# Lab 2: Linear Regression - 선형 회귀

## Theoretical Overview - 이론적 개요

$$ H(x) = Wx + b $$
$$ cost(W,b) = \cfrac{1}{m} \sum\limits_{i=1}^m (H(x^{(i)}) - y^{(i)} )^2  $$

- $H(x)$: 주어진 $x$값에 대한 예측을 하는 함수
- $cost(W, b)$: $H(x)$와 $y$사이의 오차

## Imports

In [206]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [207]:
#For reproducibility
torch.manual_seed(1)

## Data

In [208]:
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

print("---x_train---")
print(x_train)
print(x_train.shape)

print("---y_train---")
print(y_train)
print(y_train.shape)

---x_train---
tensor([[1.],
        [2.],
        [3.]])
torch.Size([3, 1])
---y_train---
tensor([[1.],
        [2.],
        [3.]])
torch.Size([3, 1])


PyTorch는 기본적으로 NCHW 형태라고 한다. 이에 관한 더 자세한 내용은 [다음](https://stackoverflow.com/questions/37689423/convert-between-nhwc-and-nchw-in-tensorflow)을 참고.


## Weight Initialization - 가중치 초기화



In [209]:
W = torch.zeros(1, requires_grad = True)
print(W)

tensor([0.], requires_grad=True)


In [210]:
b = torch.zeros(1, requires_grad= True)
print(b)

tensor([0.], requires_grad=True)


## Hypothesis - 가설 함수

$$ H(x) = Wx + b $$

In [211]:
hypothesis = x_train * W + b
print(hypothesis)

tensor([[0.],
        [0.],
        [0.]], grad_fn=<AddBackward0>)


## Cost - 손실 함수

$$ cost(W,b) = \cfrac{1}{m} \sum\limits_{i=1}^m (H(x^{(i)}) - y^{(i)})^2 $$

In [212]:
cost = torch.mean((hypothesis - y_train) ** 2)
print(cost)

tensor(4.6667, grad_fn=<MeanBackward0>)


## Gradient Descent - 경사 하강법

In [213]:
optimizer = optim.SGD([W, b], lr= 0.01)

optimizer.zero_grad()
cost.backward()
optimizer.step()

print(W)
print(b)

tensor([0.0933], requires_grad=True)
tensor([0.0400], requires_grad=True)


In [214]:
hypothesis = x_train * W + b
print(hypothesis)

tensor([[0.1333],
        [0.2267],
        [0.3200]], grad_fn=<AddBackward0>)


In [215]:
cost = torch.mean((hypothesis - y_train) ** 2)
print(cost)

tensor(3.6927, grad_fn=<MeanBackward0>)


## Training with Full Code

반복문을 이용하여 데이터셋을 여러번 훈련시키기

In [216]:
#데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

#모델 초기화
W = torch.zeros(1, requires_grad= True)
b = torch.zeros(1, requires_grad= True)

#optimizer 설정
optimizer = optim.SGD([W, b], lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
  #H(x) 계산
  hypothesis = x_train * W + b

  #cost 계산
  cost = torch.mean((hypothesis - y_train) ** 2)

  #cost를 이용하여 H(x) 개선 시키기
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  #100번 마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W.item(), b.item(), cost.item()
        ))

Epoch    0/1000 W: 0.093, b: 0.040 Cost: 4.666667
Epoch  100/1000 W: 0.873, b: 0.289 Cost: 0.012043
Epoch  200/1000 W: 0.900, b: 0.227 Cost: 0.007442
Epoch  300/1000 W: 0.921, b: 0.179 Cost: 0.004598
Epoch  400/1000 W: 0.938, b: 0.140 Cost: 0.002842
Epoch  500/1000 W: 0.951, b: 0.110 Cost: 0.001756
Epoch  600/1000 W: 0.962, b: 0.087 Cost: 0.001085
Epoch  700/1000 W: 0.970, b: 0.068 Cost: 0.000670
Epoch  800/1000 W: 0.976, b: 0.054 Cost: 0.000414
Epoch  900/1000 W: 0.981, b: 0.042 Cost: 0.000256
Epoch 1000/1000 W: 0.985, b: 0.033 Cost: 0.000158


## High-level Implementation with `nn.Module`

사용하는 데이터

In [217]:
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

PyTorch의 모든 모델은 제공되는 `nn.Module`을 상속하여 만든다. 따라서 linear regression 모델 또한 `nn.Module`을 상속받는다.

In [218]:
class LinearRegressionModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(1, 1) #하나의 입력에 대해서 하나의 출력을 가짐(입력차원, 출력차원)
  
  def forward(self, x):
    return self.linear(x)

모델의 `__init__`에서는 사용할 레이어를 정의한다. 여기서는 linear regression 모델을 만들기 때문에, `nn.Linear`를 이용한다. 그리고 `forward`에서는 이 모델이 입력을 받았을 때 어떤 출력을 계산하는지 알려준다(test).

In [219]:
model = LinearRegressionModel()

### Hypothesis
이제 모델을 생성해서 예측값 $H(x)$를 구한다.

아래 코드에서 [model(x_train)은 forward 메소드를 호출](https://jungeui.tistory.com/26)하는 듯 하다. W와 b는 model을 만들 때 [임의로 정해진다](https://wikidocs.net/55409.

In [220]:
hypothesis = model(x_train)

print(hypothesis)

tensor([[0.0739],
        [0.5891],
        [1.1044]], grad_fn=<AddmmBackward0>)


### Cost
이제 Mean Squared Error(MSE)로 cost를 구한다. MSE 역시 PyTorch에서 제공한다.

In [221]:
print(hypothesis)
print(y_train)

tensor([[0.0739],
        [0.5891],
        [1.1044]], grad_fn=<AddmmBackward0>)
tensor([[1.],
        [2.],
        [3.]])


In [222]:
cost = F.mse_loss(hypothesis, y_train)

print(cost)

tensor(2.1471, grad_fn=<MseLossBackward0>)


### Gradient Descent 
$H(x)$의 $W, b$를 바꾸어서 cost를 줄입니다. 이때 PyTorch의 `torch.optim`에 있는 `optimizer`들 중 하나를 사용할 수 있습니다.
`model.parameters()`는 $H(x)$의 $W, b$를 반환합니다.

In [223]:
optimizer = optim.SGD(model.parameters(), lr=0.01) 

optimizer.zero_grad()
cost.backward()
optimizer.step()

### Training with Full Code

In [224]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])
# 모델 초기화
model = LinearRegressionModel()
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()
        ))

Epoch    0/1000 W: -0.101, b: 0.508 Cost: 4.630286
Epoch  100/1000 W: 0.713, b: 0.653 Cost: 0.061555
Epoch  200/1000 W: 0.774, b: 0.514 Cost: 0.038037
Epoch  300/1000 W: 0.822, b: 0.404 Cost: 0.023505
Epoch  400/1000 W: 0.860, b: 0.317 Cost: 0.014525
Epoch  500/1000 W: 0.890, b: 0.250 Cost: 0.008975
Epoch  600/1000 W: 0.914, b: 0.196 Cost: 0.005546
Epoch  700/1000 W: 0.932, b: 0.154 Cost: 0.003427
Epoch  800/1000 W: 0.947, b: 0.121 Cost: 0.002118
Epoch  900/1000 W: 0.958, b: 0.095 Cost: 0.001309
Epoch 1000/1000 W: 0.967, b: 0.075 Cost: 0.000809


점점 cost가 줄어드는 것을 볼 수 있다.